In [2]:
from __future__ import print_function
from functools import reduce
import re


import numpy as np
import pandas as pd

from keras.utils.data_utils import get_file
from keras.layers.embeddings import Embedding
from keras import layers
from keras.layers import recurrent, Input, Activation, Dense, Permute, Dropout, add, dot, concatenate, LSTM
from keras.models import Sequential, Model
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [4]:
def tokenize(sent):
    '''Return the tokens of a sentence including punctuation.
    >>> tokenize('Bob dropped the apple. Where is the apple?')
    ['Bob', 'dropped', 'the', 'apple', '.', 'Where', 'is', 'the', 'apple', '?']
    '''
    return [x.strip() for x in re.split('(\W+)?', sent) if x.strip()]

In [5]:
def parse_stories(lines, only_supporting=False):
    '''Parse stories provided in the bAbi tasks format
    If only_supporting is true,
    only the sentences that support the answer are kept.
    '''
    data = []
    story = []
    for line in lines:
        line = line.strip()
        nid,line = line.split(' ',1)
        nid = int(nid)
        if nid == 1:
            story = []
        if '\t' in line:
            q, a, supporting = line.split('\t')
            q = tokenize(q)
            substory = None
            if only_supporting:
                # Only select the related substory
                supporting = map(int, supporting.split())
                substory = [story[i - 1] for i in supporting]
            else:
                # Provide all the substories
                substory = [x for x in story if x]
            data.append((substory, q, a))
            story.append('')
        else:
            sent = tokenize(line)
            story.append(sent)
    return data

In [6]:
def get_stories(f, only_supporting=False, max_length=None):
    '''Given a file name, read the file, retrieve the stories,
    and then convert the sentences into a single story.
    If max_length is supplied,
    any stories longer than max_length tokens will be discarded.
    '''
    data = parse_stories(f.readlines(), only_supporting=only_supporting)
    flatten = lambda data: reduce(lambda x, y: x + y, data)
    data = [(flatten(story), q, answer) for story, q, answer in data if not max_length or len(flatten(story)) < max_length]
    return data


In [7]:
def vectorize_stories(data, word_idx, story_maxlen, query_maxlen):
    xs = []
    xqs = []
    ys = []
    for story, query, answer in data:
        x = [word_idx[w] for w in story]
        xq = [word_idx[w] for w in query]
        # let's not forget that index 0 is reserved
        y = np.zeros(len(word_idx) + 1)
        y[word_idx[answer]] = 1
        xs.append(x)
        xqs.append(xq)
        ys.append(y)
    return pad_sequences(xs, maxlen=story_maxlen), pad_sequences(xqs, maxlen=query_maxlen), np.array(ys)

In [29]:
# Default QA1 with 1000 samples
# challenge = 'tasks_1-20_v1-2/en/qa1_single-supporting-fact_{}.txt'
# QA1 with 10,000 samples
path = ['E:/bAbI dataset tasks/tasks_1-20_v1-2/en-10k/qa1_single-supporting-fact_{}.txt',
             'E:/bAbI dataset tasks/tasks_1-20_v1-2/en-10k/qa2_two-supporting-facts_{}.txt',
             'E:/bAbI dataset tasks/tasks_1-20_v1-2/en-10k/qa3_three-supporting-facts_{}.txt',
             'E:/bAbI dataset tasks/tasks_1-20_v1-2/en-10k/qa4_two-arg-relations_{}.txt',
             'E:/bAbI dataset tasks/tasks_1-20_v1-2/en-10k/qa5_three-arg-relations_{}.txt',
             'E:/bAbI dataset tasks/tasks_1-20_v1-2/en-10k/qa6_yes-no-questions_{}.txt',
             'E:/bAbI dataset tasks/tasks_1-20_v1-2/en-10k/qa7_counting_{}.txt',
             'E:/bAbI dataset tasks/tasks_1-20_v1-2/en-10k/qa8_lists-sets_{}.txt',
             'E:/bAbI dataset tasks/tasks_1-20_v1-2/en-10k/qa9_simple-negation_{}.txt',
             'E:/bAbI dataset tasks/tasks_1-20_v1-2/en-10k/qa10_indefinite-knowledge_{}.txt',
             'E:/bAbI dataset tasks/tasks_1-20_v1-2/en-10k/qa11_basic-coreference_{}.txt',
             'E:/bAbI dataset tasks/tasks_1-20_v1-2/en-10k/qa12_conjunction_{}.txt',
             'E:/bAbI dataset tasks/tasks_1-20_v1-2/en-10k/qa13_compound-coreference_{}.txt',
             'E:/bAbI dataset tasks/tasks_1-20_v1-2/en-10k/qa14_time-reasoning_{}.txt',
             'E:/bAbI dataset tasks/tasks_1-20_v1-2/en-10k/qa15_basic-deduction_{}.txt',
             'E:/bAbI dataset tasks/tasks_1-20_v1-2/en-10k/qa16_basic-induction_{}.txt',
             'E:/bAbI dataset tasks/tasks_1-20_v1-2/en-10k/qa17_positional-reasoning_{}.txt',
             'E:/bAbI dataset tasks/tasks_1-20_v1-2/en-10k/qa18_size-reasoning_{}.txt',
             'E:/bAbI dataset tasks/tasks_1-20_v1-2/en-10k/qa19_path-finding_{}.txt',
             'E:/bAbI dataset tasks/tasks_1-20_v1-2/en-10k/qa20_agents-motivations_{}.txt']

names = ['QA1: Single supporting fact', 'QA2:Two supporting facts','QA3: Three supporting facts',
         'QA4: Two arg relations', 'QA5: Three arg relations', 'QA6: Yes No Questions', 'QA7: Counting',
         'QA8: Lists Sets', 'QA9: Simple Negation', 'QA10: Indefinite Knowledge', 'QA11: Basic Coreference',
         'QA12: Conjunction', 'QA13: Compound Co-reference', 'QA14: Time Reasoning', 'QA15: Basic Deduction',
         'QA16: Basic Induction', 'QA17: Positional Reasoning', 'QA18: Size Reasoning', 'QA19: Path Finding',
         'QA20: Agents Motivations']
# QA2 with 1000 samples
# challenge = 'D:/Major Project Anmol/bAbI/tasks_1-20_v1-2/en/qa2_two-supporting-facts_{}.txt'
# QA2 with 10,000 samples
# challenge = 'tasks_1-20_v1-2/en-10k/qa2_two-supporting-facts_{}.txt'
#with tarfile.open("D:\Major Project Anmol\bAbI\tasks_1-20_v1-2.tar.gz") as tar:


   


In [9]:
def LSTMmodel(i):
    dataset = path[i]
    train = get_stories(open(dataset.format('train'),'r'))
    test = get_stories(open(dataset.format('test'),'r'))
    vocab = set()
    for story, q, answer in train + test:
        vocab |= set(story + q + [answer])
    vocab = sorted(vocab)
    # Reserve 0 for masking via pad_sequences
    vocab_size = len(vocab) + 1
    word_idx = dict((c, i + 1) for i, c in enumerate(vocab))
    story_maxlen = max(map(len, (x for x, _, _ in train + test)))
    query_maxlen = max(map(len, (x for _, x, _ in train + test)))
    x, xq, y = vectorize_stories(train, word_idx, story_maxlen, query_maxlen)
    tx, txq, ty = vectorize_stories(test, word_idx, story_maxlen, query_maxlen)
    print('vocab = {}\n'.format(vocab))
    print('x.shape = {}\n'.format(x.shape))
    print('xq.shape = {}\n'.format(xq.shape))
    print('y.shape = {}\n'.format(y.shape))
    print('story_maxlen, query_maxlen = {}, {}\n'.format(story_maxlen, query_maxlen))
    print('An example tuple from this dataset=====> {} \n\n\n'.format(train[0]))
    RNN = recurrent.LSTM
    EMBED_HIDDEN_SIZE = 50
    SENT_HIDDEN_SIZE = 100
    QUERY_HIDDEN_SIZE = 100
    BATCH_SIZE = 32
    EPOCHS = 40
    print('RNN / Embed / Sent / Query = {}, {}, {}, {}\n\n'.format(RNN,EMBED_HIDDEN_SIZE,SENT_HIDDEN_SIZE,QUERY_HIDDEN_SIZE))
    print('Build model...\n\n')

    sentence = layers.Input(shape=(story_maxlen,), dtype='int32')
    encoded_sentence = layers.Embedding(vocab_size, EMBED_HIDDEN_SIZE)(sentence)
    encoded_sentence = layers.Dropout(0.3)(encoded_sentence)

    question = layers.Input(shape=(query_maxlen,), dtype='int32')
    encoded_question = layers.Embedding(vocab_size, EMBED_HIDDEN_SIZE)(question)
    encoded_question = layers.Dropout(0.3)(encoded_question)
    encoded_question = RNN(EMBED_HIDDEN_SIZE)(encoded_question)
    encoded_question = layers.RepeatVector(story_maxlen)(encoded_question)

    merged = layers.add([encoded_sentence, encoded_question])
    merged = RNN(EMBED_HIDDEN_SIZE)(merged)
    merged = layers.Dropout(0.3)(merged)
    preds = layers.Dense(vocab_size, activation='softmax')(merged)

    model = Model([sentence, question], preds)
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    print('Training\n\n')
    model.fit([x, xq], y,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_split=0.05)
    loss, acc = model.evaluate([tx, txq], ty,
                           batch_size=BATCH_SIZE)
    print('\n\n\n')
    print('Test loss / test accuracy = {:.4f} / {:.4f}'.format(loss, acc))
    return loss, acc



In [10]:
def memoryNet(i):
    dataset = path[i]
    train = get_stories(open(dataset.format('train'),'r'))
    test = get_stories(open(dataset.format('test'),'r'))
    vocab = set()
    for story, q, answer in train + test:
        vocab |= set(story + q + [answer])
    vocab = sorted(vocab)
    # Reserve 0 for masking via pad_sequences
    vocab_size = len(vocab) + 1
    word_idx = dict((c, i + 1) for i, c in enumerate(vocab))
    story_maxlen = max(map(len, (x for x, _, _ in train + test)))
    query_maxlen = max(map(len, (x for _, x, _ in train + test)))
    x, xq, y = vectorize_stories(train, word_idx, story_maxlen, query_maxlen)
    tx, txq, ty = vectorize_stories(test, word_idx, story_maxlen, query_maxlen)
    print('vocab = {}\n'.format(vocab))
    print('x.shape = {}\n'.format(x.shape))
    print('xq.shape = {}\n'.format(xq.shape))
    print('y.shape = {}\n'.format(y.shape))
    print('story_maxlen, query_maxlen = {}, {}\n\n\n'.format(story_maxlen, query_maxlen))
     
    input_sequence = Input((story_maxlen,))
    question = Input((query_maxlen,))
 

    input_encoder_m = Sequential()
    input_encoder_m.add(Embedding(input_dim=vocab_size,
                              output_dim=64))
    input_encoder_m.add(Dropout(0.4))
    
    
    input_encoder_c = Sequential()
    input_encoder_c.add(Embedding(input_dim=vocab_size,
                              output_dim=query_maxlen))
    input_encoder_c.add(Dropout(0.4))
      
        

    question_encoder = Sequential()
    question_encoder.add(Embedding(input_dim=vocab_size,
                               output_dim=64,
                               input_length=query_maxlen))
    question_encoder.add(Dropout(0.4))
    
    
    input_encoded_m = input_encoder_m(input_sequence)
    input_encoded_c = input_encoder_c(input_sequence)
    question_encoded = question_encoder(question)
 

    match = dot([input_encoded_m, question_encoded], axes=(2, 2))
    match = Activation('softmax')(match)

    
    response = add([match, input_encoded_c])  
    response = Permute((2, 1))(response)  

    
    answer = concatenate([response, question_encoded])

     
    answer = LSTM(32)(answer)   
 
   
    answer = Dropout(0.4)(answer)
    answer = Dense(vocab_size)(answer)   
    
    answer = Activation('softmax')(answer)

     
    model = Model([input_sequence, question], answer)
    model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

  
    test_y = np.ones(ty.shape[0])
    print(test_y.shape)
    for ix in range(test_y.shape[0]):
        test_y[ix] = np.argmax(ty[ix])
    
    train_y = np.ones(y.shape[0])
    print(train_y.shape)
    for ix in range(train_y.shape[0]):
        train_y[ix] = np.argmax(y[ix])
    
    
    model.fit([x, xq], train_y,
          batch_size=32,
          epochs=120,
          validation_data=([tx, txq], test_y))
    
    loss, acc = model.evaluate([tx, txq], test_y,
                           batch_size=32)
    print('\n\n\n')
    print('Test loss / test accuracy = {:.4f} / {:.4f}'.format(loss, acc))
    return loss, acc

In [11]:
resultsLSTM = pd.DataFrame(columns=['QA Dataset','Loss','Accuracy'])
resultsMem = pd.DataFrame(columns=['QA Dataset','Loss','Accuracy'])

In [12]:
def append(model,loss,acc,i):
    if model =='LSTM':
        resultsLSTM.loc[i] = [names[i],loss,acc]
    else:
        resultsMem.loc[i] = [names[i],loss,acc]

# QA1 One Supporting Fact

In [14]:
i=0
loss, acc = LSTMmodel(i)
append('LSTM',loss,acc,i)
print('\n\n\n')
loss, acc = memoryNet(i)
append('Memory',loss,acc,i)
i+=1

C:\Users\Manik\Anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


vocab = ['.', '?', 'Daniel', 'John', 'Mary', 'Sandra', 'Where', 'back', 'bathroom', 'bedroom', 'garden', 'hallway', 'is', 'journeyed', 'kitchen', 'moved', 'office', 'the', 'to', 'travelled', 'went']

x.shape = (10000, 68)

xq.shape = (10000, 4)

y.shape = (10000, 22)

story_maxlen, query_maxlen = 68, 4

An example tuple from this dataset=====> (['Mary', 'moved', 'to', 'the', 'bathroom', '.', 'John', 'went', 'to', 'the', 'hallway', '.'], ['Where', 'is', 'Mary', '?'], 'bathroom') 



RNN / Embed / Sent / Query = <class 'keras.layers.recurrent.LSTM'>, 50, 100, 100


Build model...


Training


Train on 9500 samples, validate on 500 samples
Epoch 1/40
9500/9500 [==============================] - 21s 2ms/step - loss: 1.9519 - acc: 0.1834 - val_loss: 1.6867 - val_acc: 0.4240
Epoch 2/40
9500/9500 [==============================] - 17s 2ms/step - loss: 1.4237 - acc: 0.4706 - val_loss: 1.2967 - val_acc: 0.5040
Epoch 3/40
9500/9500 [==============================] - 18s 2ms/step - loss: 1.2322 -

Epoch 13/120
10000/10000 [==============================] - 4s 448us/step - loss: 1.3840 - acc: 0.4779 - val_loss: 1.3360 - val_acc: 0.4990
Epoch 14/120
10000/10000 [==============================] - 4s 448us/step - loss: 1.3614 - acc: 0.4851 - val_loss: 1.3260 - val_acc: 0.4970
Epoch 15/120
10000/10000 [==============================] - 4s 447us/step - loss: 1.3654 - acc: 0.4850 - val_loss: 1.3170 - val_acc: 0.5040
Epoch 16/120
10000/10000 [==============================] - 5s 474us/step - loss: 1.3503 - acc: 0.4834 - val_loss: 1.3141 - val_acc: 0.5060
Epoch 17/120
10000/10000 [==============================] - 4s 444us/step - loss: 1.3449 - acc: 0.4920 - val_loss: 1.3136 - val_acc: 0.5040
Epoch 18/120
10000/10000 [==============================] - 4s 448us/step - loss: 1.3408 - acc: 0.4905 - val_loss: 1.2997 - val_acc: 0.5140
Epoch 19/120
10000/10000 [==============================] - 4s 448us/step - loss: 1.3216 - acc: 0.5002 - val_loss: 1.2908 - val_acc: 0.5130
Epoch 20/120
10000/1

10000/10000 [==============================] - 4s 449us/step - loss: 0.5632 - acc: 0.7893 - val_loss: 0.5662 - val_acc: 0.7720
Epoch 72/120
10000/10000 [==============================] - 4s 445us/step - loss: 0.5352 - acc: 0.7981 - val_loss: 0.5542 - val_acc: 0.7730
Epoch 73/120
10000/10000 [==============================] - 4s 448us/step - loss: 0.5230 - acc: 0.8031 - val_loss: 0.5190 - val_acc: 0.7950
Epoch 74/120
10000/10000 [==============================] - 4s 450us/step - loss: 0.4933 - acc: 0.8125 - val_loss: 0.5059 - val_acc: 0.7990
Epoch 75/120
10000/10000 [==============================] - 4s 447us/step - loss: 0.4781 - acc: 0.8146 - val_loss: 0.4739 - val_acc: 0.8060
Epoch 76/120
10000/10000 [==============================] - 4s 446us/step - loss: 0.4586 - acc: 0.8253 - val_loss: 0.4667 - val_acc: 0.8110
Epoch 77/120
10000/10000 [==============================] - 4s 447us/step - loss: 0.4468 - acc: 0.8292 - val_loss: 0.4478 - val_acc: 0.8250
Epoch 78/120
10000/10000 [=======

# QA2 Two Supporting Fact

In [15]:
loss, acc = LSTMmodel(i)
append('LSTM',loss,acc,i)
print('\n\n\n')
loss, acc = memoryNet(i)
append('Memory',loss,acc,i)
i+=1

C:\Users\Manik\Anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


vocab = ['.', '?', 'Daniel', 'John', 'Mary', 'Sandra', 'Where', 'apple', 'back', 'bathroom', 'bedroom', 'discarded', 'down', 'dropped', 'football', 'garden', 'got', 'grabbed', 'hallway', 'is', 'journeyed', 'kitchen', 'left', 'milk', 'moved', 'office', 'picked', 'put', 'the', 'there', 'to', 'took', 'travelled', 'up', 'went']

x.shape = (10000, 552)

xq.shape = (10000, 5)

y.shape = (10000, 36)

story_maxlen, query_maxlen = 552, 5

An example tuple from this dataset=====> (['Mary', 'moved', 'to', 'the', 'bathroom', '.', 'Sandra', 'journeyed', 'to', 'the', 'bedroom', '.', 'Mary', 'got', 'the', 'football', 'there', '.', 'John', 'went', 'to', 'the', 'kitchen', '.', 'Mary', 'went', 'back', 'to', 'the', 'kitchen', '.', 'Mary', 'went', 'back', 'to', 'the', 'garden', '.'], ['Where', 'is', 'the', 'football', '?'], 'garden') 



RNN / Embed / Sent / Query = <class 'keras.layers.recurrent.LSTM'>, 50, 100, 100


Build model...


Training


Train on 9500 samples, validate on 500 samples
Epoch 1/40
9

Epoch 9/120
10000/10000 [==============================] - 22s 2ms/step - loss: 1.6825 - acc: 0.2943 - val_loss: 1.6482 - val_acc: 0.3280
Epoch 10/120
10000/10000 [==============================] - 23s 2ms/step - loss: 1.6808 - acc: 0.2987 - val_loss: 1.6542 - val_acc: 0.3010
Epoch 11/120
10000/10000 [==============================] - 22s 2ms/step - loss: 1.6580 - acc: 0.3173 - val_loss: 1.6380 - val_acc: 0.3370
Epoch 12/120
10000/10000 [==============================] - 22s 2ms/step - loss: 1.6560 - acc: 0.3203 - val_loss: 1.6377 - val_acc: 0.3210
Epoch 13/120
10000/10000 [==============================] - 23s 2ms/step - loss: 1.6421 - acc: 0.3331 - val_loss: 1.6444 - val_acc: 0.3290
Epoch 14/120
10000/10000 [==============================] - 22s 2ms/step - loss: 1.6458 - acc: 0.3263 - val_loss: 1.6457 - val_acc: 0.3230
Epoch 15/120
10000/10000 [==============================] - 23s 2ms/step - loss: 1.6286 - acc: 0.3328 - val_loss: 1.6096 - val_acc: 0.3430
Epoch 16/120
10000/10000 [==

Epoch 68/120
10000/10000 [==============================] - 23s 2ms/step - loss: 1.2568 - acc: 0.5154 - val_loss: 1.7203 - val_acc: 0.3440
Epoch 69/120
10000/10000 [==============================] - 22s 2ms/step - loss: 1.2534 - acc: 0.5183 - val_loss: 1.7213 - val_acc: 0.3480
Epoch 70/120
10000/10000 [==============================] - 23s 2ms/step - loss: 1.2459 - acc: 0.5162 - val_loss: 1.7607 - val_acc: 0.3450
Epoch 71/120
10000/10000 [==============================] - 22s 2ms/step - loss: 1.2276 - acc: 0.5318 - val_loss: 1.7517 - val_acc: 0.3300
Epoch 72/120
10000/10000 [==============================] - 23s 2ms/step - loss: 1.2256 - acc: 0.5275 - val_loss: 1.7619 - val_acc: 0.3510
Epoch 73/120
10000/10000 [==============================] - 23s 2ms/step - loss: 1.2306 - acc: 0.5219 - val_loss: 1.7736 - val_acc: 0.3510
Epoch 74/120
10000/10000 [==============================] - 23s 2ms/step - loss: 1.2318 - acc: 0.5244 - val_loss: 1.7889 - val_acc: 0.3360
Epoch 75/120
10000/10000 [=

# QA3 Three Supporting Fact

In [16]:
loss, acc = LSTMmodel(i)
append('LSTM',loss,acc,i)
print('\n\n\n')
loss, acc = memoryNet(i)
append('Memory',loss,acc,i)
i+=1

C:\Users\Manik\Anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


vocab = ['.', '?', 'Daniel', 'John', 'Mary', 'Sandra', 'Where', 'apple', 'back', 'bathroom', 'bedroom', 'before', 'discarded', 'down', 'dropped', 'football', 'garden', 'got', 'grabbed', 'hallway', 'journeyed', 'kitchen', 'left', 'milk', 'moved', 'office', 'picked', 'put', 'the', 'there', 'to', 'took', 'travelled', 'up', 'was', 'went']

x.shape = (10000, 1875)

xq.shape = (10000, 8)

y.shape = (10000, 37)

story_maxlen, query_maxlen = 1875, 8

An example tuple from this dataset=====> (['Mary', 'moved', 'to', 'the', 'bathroom', '.', 'Sandra', 'journeyed', 'to', 'the', 'bedroom', '.', 'Mary', 'got', 'the', 'football', 'there', '.', 'John', 'went', 'back', 'to', 'the', 'bedroom', '.', 'Mary', 'journeyed', 'to', 'the', 'office', '.', 'John', 'journeyed', 'to', 'the', 'office', '.', 'John', 'took', 'the', 'milk', '.', 'Daniel', 'went', 'back', 'to', 'the', 'kitchen', '.', 'John', 'moved', 'to', 'the', 'bedroom', '.', 'Daniel', 'went', 'back', 'to', 'the', 'hallway', '.', 'Daniel', 'took', 't

10000/10000 [==============================] - 78s 8ms/step - loss: 1.8053 - acc: 0.1966 - val_loss: 1.7471 - val_acc: 0.2020
Epoch 3/120
10000/10000 [==============================] - 78s 8ms/step - loss: 1.7467 - acc: 0.2034 - val_loss: 1.6932 - val_acc: 0.2200
Epoch 4/120
10000/10000 [==============================] - 78s 8ms/step - loss: 1.7086 - acc: 0.2163 - val_loss: 1.6825 - val_acc: 0.2050
Epoch 5/120
10000/10000 [==============================] - 78s 8ms/step - loss: 1.6678 - acc: 0.2456 - val_loss: 1.6617 - val_acc: 0.2180
Epoch 6/120
10000/10000 [==============================] - 78s 8ms/step - loss: 1.6308 - acc: 0.2659 - val_loss: 1.6552 - val_acc: 0.2120
Epoch 7/120
10000/10000 [==============================] - 79s 8ms/step - loss: 1.6039 - acc: 0.2977 - val_loss: 1.6587 - val_acc: 0.2210
Epoch 8/120
10000/10000 [==============================] - 78s 8ms/step - loss: 1.5762 - acc: 0.3075 - val_loss: 1.6412 - val_acc: 0.2340
Epoch 9/120
10000/10000 [=====================

10000/10000 [==============================] - 78s 8ms/step - loss: 0.8407 - acc: 0.6721 - val_loss: 2.2420 - val_acc: 0.2410
Epoch 62/120
10000/10000 [==============================] - 78s 8ms/step - loss: 0.8328 - acc: 0.6815 - val_loss: 2.2512 - val_acc: 0.2470
Epoch 63/120
10000/10000 [==============================] - 78s 8ms/step - loss: 0.8190 - acc: 0.6826 - val_loss: 2.3513 - val_acc: 0.2330
Epoch 64/120
10000/10000 [==============================] - 78s 8ms/step - loss: 0.8200 - acc: 0.6839 - val_loss: 2.3354 - val_acc: 0.2390
Epoch 65/120
10000/10000 [==============================] - 79s 8ms/step - loss: 0.8237 - acc: 0.6858 - val_loss: 2.3003 - val_acc: 0.2570
Epoch 66/120
10000/10000 [==============================] - 78s 8ms/step - loss: 0.8231 - acc: 0.6833 - val_loss: 2.2164 - val_acc: 0.2560
Epoch 67/120
10000/10000 [==============================] - 79s 8ms/step - loss: 0.7879 - acc: 0.6938 - val_loss: 2.3235 - val_acc: 0.2510
Epoch 68/120
10000/10000 [==============

Epoch 120/120
1000/1000 [==============================] - 2s 2ms/step




Test loss / test accuracy = 2.7129 / 0.2430


# QA4 Two Argument Relations

In [18]:
loss, acc = LSTMmodel(i)
append('LSTM',loss,acc,i)
print('\n\n\n')
loss, acc = memoryNet(i)
append('Memory',loss,acc,i)
i+=1

C:\Users\Manik\Anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


vocab = ['.', '?', 'The', 'What', 'bathroom', 'bedroom', 'east', 'garden', 'hallway', 'is', 'kitchen', 'north', 'of', 'office', 'south', 'the', 'west']

x.shape = (10000, 16)

xq.shape = (10000, 7)

y.shape = (10000, 18)

story_maxlen, query_maxlen = 16, 7

An example tuple from this dataset=====> (['The', 'office', 'is', 'north', 'of', 'the', 'kitchen', '.', 'The', 'garden', 'is', 'south', 'of', 'the', 'kitchen', '.'], ['What', 'is', 'north', 'of', 'the', 'kitchen', '?'], 'office') 



RNN / Embed / Sent / Query = <class 'keras.layers.recurrent.LSTM'>, 50, 100, 100


Build model...


Training


Train on 9500 samples, validate on 500 samples
Epoch 1/40
9500/9500 [==============================] - 10s 1ms/step - loss: 1.9537 - acc: 0.1666 - val_loss: 1.8017 - val_acc: 0.2100
Epoch 2/40
9500/9500 [==============================] - 6s 684us/step - loss: 1.7795 - acc: 0.2008 - val_loss: 1.6567 - val_acc: 0.2680
Epoch 3/40
9500/9500 [==============================] - 7s 692us/step - loss: 1

Epoch 13/120
10000/10000 [==============================] - 4s 368us/step - loss: 0.8482 - acc: 0.6481 - val_loss: 0.6205 - val_acc: 0.7210
Epoch 14/120
10000/10000 [==============================] - 4s 402us/step - loss: 0.8190 - acc: 0.6576 - val_loss: 0.5695 - val_acc: 0.7450
Epoch 15/120
10000/10000 [==============================] - 4s 373us/step - loss: 0.7751 - acc: 0.6760 - val_loss: 0.5505 - val_acc: 0.7370
Epoch 16/120
10000/10000 [==============================] - 4s 372us/step - loss: 0.7632 - acc: 0.6749 - val_loss: 0.5482 - val_acc: 0.7260
Epoch 17/120
10000/10000 [==============================] - 4s 372us/step - loss: 0.7336 - acc: 0.6880 - val_loss: 0.5554 - val_acc: 0.7390
Epoch 18/120
10000/10000 [==============================] - 4s 373us/step - loss: 0.7036 - acc: 0.7003 - val_loss: 0.5242 - val_acc: 0.7510
Epoch 19/120
10000/10000 [==============================] - 4s 372us/step - loss: 0.6911 - acc: 0.6987 - val_loss: 0.4859 - val_acc: 0.7570
Epoch 20/120
10000/1

10000/10000 [==============================] - 4s 366us/step - loss: 0.3938 - acc: 0.7828 - val_loss: 0.3487 - val_acc: 0.7680
Epoch 72/120
10000/10000 [==============================] - 4s 369us/step - loss: 0.3941 - acc: 0.7849 - val_loss: 0.3601 - val_acc: 0.7750
Epoch 73/120
10000/10000 [==============================] - 4s 371us/step - loss: 0.3863 - acc: 0.7915 - val_loss: 0.3418 - val_acc: 0.7860
Epoch 74/120
10000/10000 [==============================] - 4s 402us/step - loss: 0.3905 - acc: 0.7887 - val_loss: 0.3436 - val_acc: 0.7680
Epoch 75/120
10000/10000 [==============================] - 4s 375us/step - loss: 0.3894 - acc: 0.7934 - val_loss: 0.3413 - val_acc: 0.7730
Epoch 76/120
10000/10000 [==============================] - 4s 373us/step - loss: 0.3874 - acc: 0.7830 - val_loss: 0.3443 - val_acc: 0.7780
Epoch 77/120
10000/10000 [==============================] - 4s 373us/step - loss: 0.3872 - acc: 0.7856 - val_loss: 0.3480 - val_acc: 0.7700
Epoch 78/120
10000/10000 [=======

# QA5 Three Argument Relations

In [19]:
loss, acc = LSTMmodel(i)
append('LSTM',loss,acc,i)
print('\n\n\n')
loss, acc = memoryNet(i)
append('Memory',loss,acc,i)
i+=1

C:\Users\Manik\Anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


vocab = ['.', '?', 'Bill', 'Fred', 'Jeff', 'Mary', 'What', 'Who', 'apple', 'back', 'bathroom', 'bedroom', 'did', 'discarded', 'down', 'dropped', 'football', 'garden', 'gave', 'give', 'got', 'grabbed', 'hallway', 'handed', 'journeyed', 'kitchen', 'left', 'milk', 'moved', 'office', 'passed', 'picked', 'put', 'received', 'the', 'there', 'to', 'took', 'travelled', 'up', 'went']

x.shape = (10000, 782)

xq.shape = (10000, 8)

y.shape = (10000, 42)

story_maxlen, query_maxlen = 782, 8

An example tuple from this dataset=====> (['Bill', 'travelled', 'to', 'the', 'office', '.', 'Bill', 'picked', 'up', 'the', 'football', 'there', '.', 'Bill', 'went', 'to', 'the', 'bedroom', '.', 'Bill', 'gave', 'the', 'football', 'to', 'Fred', '.'], ['What', 'did', 'Bill', 'give', 'to', 'Fred', '?'], 'football') 



RNN / Embed / Sent / Query = <class 'keras.layers.recurrent.LSTM'>, 50, 100, 100


Build model...


Training


Train on 9500 samples, validate on 500 samples
Epoch 1/40
9500/9500 [==================

10000/10000 [==============================] - 35s 4ms/step - loss: 0.8271 - acc: 0.6446 - val_loss: 0.7930 - val_acc: 0.6690
Epoch 9/120
10000/10000 [==============================] - 35s 3ms/step - loss: 0.7847 - acc: 0.6688 - val_loss: 0.7411 - val_acc: 0.6970
Epoch 10/120
10000/10000 [==============================] - 35s 4ms/step - loss: 0.7475 - acc: 0.6918 - val_loss: 0.6889 - val_acc: 0.7180
Epoch 11/120
10000/10000 [==============================] - 35s 4ms/step - loss: 0.7188 - acc: 0.7082 - val_loss: 0.6793 - val_acc: 0.7270
Epoch 12/120
10000/10000 [==============================] - 35s 3ms/step - loss: 0.6853 - acc: 0.7185 - val_loss: 0.6427 - val_acc: 0.7270
Epoch 13/120
10000/10000 [==============================] - 35s 4ms/step - loss: 0.6437 - acc: 0.7373 - val_loss: 0.5937 - val_acc: 0.7600
Epoch 14/120
10000/10000 [==============================] - 36s 4ms/step - loss: 0.6071 - acc: 0.7568 - val_loss: 0.5505 - val_acc: 0.7810
Epoch 15/120
10000/10000 [===============

10000/10000 [==============================] - 35s 4ms/step - loss: 0.1939 - acc: 0.9333 - val_loss: 0.4219 - val_acc: 0.8710
Epoch 68/120
10000/10000 [==============================] - 35s 3ms/step - loss: 0.1832 - acc: 0.9380 - val_loss: 0.4292 - val_acc: 0.8730
Epoch 69/120
10000/10000 [==============================] - 35s 4ms/step - loss: 0.1927 - acc: 0.9353 - val_loss: 0.4812 - val_acc: 0.8610
Epoch 70/120
10000/10000 [==============================] - 35s 3ms/step - loss: 0.1851 - acc: 0.9356 - val_loss: 0.4591 - val_acc: 0.8650
Epoch 71/120
10000/10000 [==============================] - 35s 4ms/step - loss: 0.1862 - acc: 0.9382 - val_loss: 0.4660 - val_acc: 0.8640
Epoch 72/120
10000/10000 [==============================] - 35s 4ms/step - loss: 0.2070 - acc: 0.9291 - val_loss: 0.4094 - val_acc: 0.8750
Epoch 73/120
10000/10000 [==============================] - 35s 3ms/step - loss: 0.1829 - acc: 0.9375 - val_loss: 0.4232 - val_acc: 0.8730
Epoch 74/120
10000/10000 [==============

# QA6 Yes No Questions

In [20]:
loss, acc = LSTMmodel(i)
append('LSTM',loss,acc,i)
print('\n\n\n')
loss, acc = memoryNet(i)
append('Memory',loss,acc,i)
i+=1

C:\Users\Manik\Anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


vocab = ['.', '?', 'Daniel', 'Is', 'John', 'Mary', 'Sandra', 'apple', 'back', 'bathroom', 'bedroom', 'discarded', 'down', 'dropped', 'football', 'garden', 'got', 'grabbed', 'hallway', 'in', 'journeyed', 'kitchen', 'left', 'milk', 'moved', 'no', 'office', 'picked', 'put', 'the', 'there', 'to', 'took', 'travelled', 'up', 'went', 'yes']

x.shape = (10000, 156)

xq.shape = (10000, 6)

y.shape = (10000, 38)

story_maxlen, query_maxlen = 156, 6

An example tuple from this dataset=====> (['Mary', 'moved', 'to', 'the', 'bathroom', '.', 'Sandra', 'journeyed', 'to', 'the', 'bedroom', '.'], ['Is', 'Sandra', 'in', 'the', 'hallway', '?'], 'no') 



RNN / Embed / Sent / Query = <class 'keras.layers.recurrent.LSTM'>, 50, 100, 100


Build model...


Training


Train on 9500 samples, validate on 500 samples
Epoch 1/40
9500/9500 [==============================] - 37s 4ms/step - loss: 0.9655 - acc: 0.5062 - val_loss: 0.7210 - val_acc: 0.4800
Epoch 2/40
9500/9500 [==============================] - 32s 3ms

10000/10000 [==============================] - 11s 1ms/step - loss: 0.6936 - acc: 0.5084 - val_loss: 0.6939 - val_acc: 0.4760
Epoch 11/120
10000/10000 [==============================] - 10s 1ms/step - loss: 0.6932 - acc: 0.5036 - val_loss: 0.6984 - val_acc: 0.5030
Epoch 12/120
10000/10000 [==============================] - 10s 1ms/step - loss: 0.6932 - acc: 0.5158 - val_loss: 0.6939 - val_acc: 0.4970
Epoch 13/120
10000/10000 [==============================] - 10s 1ms/step - loss: 0.6922 - acc: 0.5259 - val_loss: 0.6941 - val_acc: 0.4950
Epoch 14/120
10000/10000 [==============================] - 10s 1ms/step - loss: 0.6905 - acc: 0.5344 - val_loss: 0.6911 - val_acc: 0.5300
Epoch 15/120
10000/10000 [==============================] - 11s 1ms/step - loss: 0.6832 - acc: 0.5474 - val_loss: 0.6763 - val_acc: 0.5740
Epoch 16/120
10000/10000 [==============================] - 10s 1ms/step - loss: 0.6622 - acc: 0.5985 - val_loss: 0.6541 - val_acc: 0.6230
Epoch 17/120
10000/10000 [==============

10000/10000 [==============================] - 10s 1ms/step - loss: 0.3566 - acc: 0.8483 - val_loss: 0.4019 - val_acc: 0.8130
Epoch 70/120
10000/10000 [==============================] - 10s 1ms/step - loss: 0.3564 - acc: 0.8464 - val_loss: 0.4029 - val_acc: 0.8130
Epoch 71/120
10000/10000 [==============================] - 10s 1ms/step - loss: 0.3528 - acc: 0.8484 - val_loss: 0.4140 - val_acc: 0.8130
Epoch 72/120
10000/10000 [==============================] - 10s 997us/step - loss: 0.3483 - acc: 0.8512 - val_loss: 0.4160 - val_acc: 0.8120
Epoch 73/120
10000/10000 [==============================] - 10s 998us/step - loss: 0.3404 - acc: 0.8531 - val_loss: 0.4133 - val_acc: 0.8150
Epoch 74/120
10000/10000 [==============================] - 10s 1ms/step - loss: 0.3440 - acc: 0.8534 - val_loss: 0.4105 - val_acc: 0.8210
Epoch 75/120
10000/10000 [==============================] - 10s 996us/step - loss: 0.3446 - acc: 0.8562 - val_loss: 0.4316 - val_acc: 0.8160
Epoch 76/120
10000/10000 [========

# QA7 Counting Test

In [21]:
loss, acc = LSTMmodel(i)
append('LSTM',loss,acc,i)
print('\n\n\n')
loss, acc = memoryNet(i)
append('Memory',loss,acc,i)
i+=1

C:\Users\Manik\Anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


vocab = ['.', '?', 'Daniel', 'How', 'John', 'Mary', 'Sandra', 'apple', 'back', 'bathroom', 'bedroom', 'carrying', 'discarded', 'down', 'dropped', 'football', 'garden', 'gave', 'got', 'grabbed', 'hallway', 'handed', 'is', 'journeyed', 'kitchen', 'left', 'many', 'milk', 'moved', 'none', 'objects', 'office', 'one', 'passed', 'picked', 'put', 'the', 'there', 'three', 'to', 'took', 'travelled', 'two', 'up', 'went']

x.shape = (10000, 321)

xq.shape = (10000, 7)

y.shape = (10000, 46)

story_maxlen, query_maxlen = 321, 7

An example tuple from this dataset=====> (['Mary', 'moved', 'to', 'the', 'bathroom', '.', 'Sandra', 'journeyed', 'to', 'the', 'bedroom', '.', 'John', 'went', 'to', 'the', 'kitchen', '.', 'Mary', 'took', 'the', 'football', 'there', '.'], ['How', 'many', 'objects', 'is', 'Mary', 'carrying', '?'], 'one') 



RNN / Embed / Sent / Query = <class 'keras.layers.recurrent.LSTM'>, 50, 100, 100


Build model...


Training


Train on 9500 samples, validate on 500 samples
Epoch 1/40
95

Epoch 9/120
10000/10000 [==============================] - 18s 2ms/step - loss: 0.5029 - acc: 0.7909 - val_loss: 0.4872 - val_acc: 0.7940
Epoch 10/120
10000/10000 [==============================] - 18s 2ms/step - loss: 0.4914 - acc: 0.7919 - val_loss: 0.4843 - val_acc: 0.7900
Epoch 11/120
10000/10000 [==============================] - 18s 2ms/step - loss: 0.4815 - acc: 0.7922 - val_loss: 0.5372 - val_acc: 0.7680
Epoch 12/120
10000/10000 [==============================] - 18s 2ms/step - loss: 0.4668 - acc: 0.8010 - val_loss: 0.4978 - val_acc: 0.7970
Epoch 13/120
10000/10000 [==============================] - 18s 2ms/step - loss: 0.4659 - acc: 0.7966 - val_loss: 0.4675 - val_acc: 0.7980
Epoch 14/120
10000/10000 [==============================] - 18s 2ms/step - loss: 0.4576 - acc: 0.8014 - val_loss: 0.4652 - val_acc: 0.7850
Epoch 15/120
10000/10000 [==============================] - 18s 2ms/step - loss: 0.4552 - acc: 0.8000 - val_loss: 0.4610 - val_acc: 0.8000
Epoch 16/120
10000/10000 [==

10000/10000 [==============================] - 18s 2ms/step - loss: 0.3448 - acc: 0.8415 - val_loss: 0.5231 - val_acc: 0.7820
Epoch 68/120
10000/10000 [==============================] - 18s 2ms/step - loss: 0.3406 - acc: 0.8434 - val_loss: 0.5871 - val_acc: 0.8070
Epoch 69/120
10000/10000 [==============================] - 18s 2ms/step - loss: 0.3446 - acc: 0.8448 - val_loss: 0.5198 - val_acc: 0.7880
Epoch 70/120
10000/10000 [==============================] - 18s 2ms/step - loss: 0.3451 - acc: 0.8442 - val_loss: 0.5120 - val_acc: 0.7760
Epoch 71/120
10000/10000 [==============================] - 18s 2ms/step - loss: 0.3391 - acc: 0.8507 - val_loss: 0.5431 - val_acc: 0.7880
Epoch 72/120
10000/10000 [==============================] - 18s 2ms/step - loss: 0.3406 - acc: 0.8474 - val_loss: 0.5506 - val_acc: 0.8040
Epoch 73/120
10000/10000 [==============================] - 18s 2ms/step - loss: 0.3394 - acc: 0.8496 - val_loss: 0.5596 - val_acc: 0.7930
Epoch 74/120
10000/10000 [==============

# QA8 List Sets

In [22]:
loss, acc = LSTMmodel(i)
append('LSTM',loss,acc,i)
print('\n\n\n')
loss, acc = memoryNet(i)
append('Memory',loss,acc,i)
i+=1

C:\Users\Manik\Anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


vocab = ['.', '?', 'Daniel', 'John', 'Mary', 'Sandra', 'What', 'apple', 'apple,football', 'apple,football,milk', 'apple,milk', 'apple,milk,football', 'back', 'bathroom', 'bedroom', 'carrying', 'discarded', 'down', 'dropped', 'football', 'football,apple', 'football,apple,milk', 'football,milk', 'football,milk,apple', 'garden', 'got', 'grabbed', 'hallway', 'is', 'journeyed', 'kitchen', 'left', 'milk', 'milk,apple', 'milk,apple,football', 'milk,football', 'milk,football,apple', 'moved', 'nothing', 'office', 'picked', 'put', 'the', 'there', 'to', 'took', 'travelled', 'up', 'went']

x.shape = (10000, 358)

xq.shape = (10000, 5)

y.shape = (10000, 50)

story_maxlen, query_maxlen = 358, 5

An example tuple from this dataset=====> (['Mary', 'moved', 'to', 'the', 'bathroom', '.', 'Sandra', 'journeyed', 'to', 'the', 'bedroom', '.', 'Mary', 'got', 'the', 'football', 'there', '.', 'John', 'went', 'to', 'the', 'kitchen', '.'], ['What', 'is', 'Mary', 'carrying', '?'], 'football') 



RNN / Embed / S

10000/10000 [==============================] - 17s 2ms/step - loss: 1.1517 - acc: 0.6588 - val_loss: 1.0857 - val_acc: 0.7100
Epoch 7/120
10000/10000 [==============================] - 17s 2ms/step - loss: 1.0779 - acc: 0.6908 - val_loss: 0.9972 - val_acc: 0.7150
Epoch 8/120
10000/10000 [==============================] - 17s 2ms/step - loss: 1.0178 - acc: 0.7123 - val_loss: 0.8885 - val_acc: 0.7480
Epoch 9/120
10000/10000 [==============================] - 17s 2ms/step - loss: 0.9596 - acc: 0.7321 - val_loss: 0.8624 - val_acc: 0.7520
Epoch 10/120
10000/10000 [==============================] - 17s 2ms/step - loss: 0.9392 - acc: 0.7401 - val_loss: 0.8368 - val_acc: 0.7490
Epoch 11/120
10000/10000 [==============================] - 17s 2ms/step - loss: 0.9077 - acc: 0.7443 - val_loss: 0.8268 - val_acc: 0.7450
Epoch 12/120
10000/10000 [==============================] - 17s 2ms/step - loss: 0.8866 - acc: 0.7483 - val_loss: 0.8318 - val_acc: 0.7370
Epoch 13/120
10000/10000 [=================

10000/10000 [==============================] - 17s 2ms/step - loss: 0.6309 - acc: 0.7974 - val_loss: 0.7511 - val_acc: 0.7560
Epoch 66/120
10000/10000 [==============================] - 17s 2ms/step - loss: 0.6090 - acc: 0.7962 - val_loss: 0.7610 - val_acc: 0.7590
Epoch 67/120
10000/10000 [==============================] - 17s 2ms/step - loss: 0.6204 - acc: 0.7971 - val_loss: 0.7135 - val_acc: 0.7560
Epoch 68/120
10000/10000 [==============================] - 17s 2ms/step - loss: 0.6253 - acc: 0.7941 - val_loss: 0.7332 - val_acc: 0.7630
Epoch 69/120
10000/10000 [==============================] - 17s 2ms/step - loss: 0.6209 - acc: 0.7958 - val_loss: 0.7186 - val_acc: 0.7580
Epoch 70/120
10000/10000 [==============================] - 17s 2ms/step - loss: 0.6165 - acc: 0.7993 - val_loss: 0.7790 - val_acc: 0.7620
Epoch 71/120
10000/10000 [==============================] - 17s 2ms/step - loss: 0.6129 - acc: 0.8004 - val_loss: 0.7634 - val_acc: 0.7570
Epoch 72/120
10000/10000 [==============

# QA9 Simple Negation

In [23]:
loss, acc = LSTMmodel(i)
append('LSTM',loss,acc,i)
print('\n\n\n')
loss, acc = memoryNet(i)
append('Memory',loss,acc,i)
i+=1

C:\Users\Manik\Anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


vocab = ['.', '?', 'Daniel', 'Is', 'John', 'Mary', 'Sandra', 'back', 'bathroom', 'bedroom', 'garden', 'hallway', 'in', 'is', 'journeyed', 'kitchen', 'longer', 'moved', 'no', 'not', 'office', 'the', 'to', 'travelled', 'went', 'yes']

x.shape = (10000, 74)

xq.shape = (10000, 6)

y.shape = (10000, 27)

story_maxlen, query_maxlen = 74, 6

An example tuple from this dataset=====> (['Mary', 'is', 'no', 'longer', 'in', 'the', 'bedroom', '.', 'Daniel', 'moved', 'to', 'the', 'hallway', '.'], ['Is', 'Mary', 'in', 'the', 'bedroom', '?'], 'no') 



RNN / Embed / Sent / Query = <class 'keras.layers.recurrent.LSTM'>, 50, 100, 100


Build model...


Training


Train on 9500 samples, validate on 500 samples
Epoch 1/40
9500/9500 [==============================] - 23s 2ms/step - loss: 0.8586 - acc: 0.5872 - val_loss: 0.6640 - val_acc: 0.6260
Epoch 2/40
9500/9500 [==============================] - 18s 2ms/step - loss: 0.6794 - acc: 0.6117 - val_loss: 0.6634 - val_acc: 0.6260
Epoch 3/40
9500/9500 [======

10000/10000 [==============================] - 6s 626us/step - loss: 0.6206 - acc: 0.6346 - val_loss: 0.6271 - val_acc: 0.6200
Epoch 13/120
10000/10000 [==============================] - 6s 625us/step - loss: 0.6189 - acc: 0.6391 - val_loss: 0.6285 - val_acc: 0.6260
Epoch 14/120
10000/10000 [==============================] - 6s 627us/step - loss: 0.6174 - acc: 0.6442 - val_loss: 0.6289 - val_acc: 0.6310
Epoch 15/120
10000/10000 [==============================] - 6s 626us/step - loss: 0.6186 - acc: 0.6450 - val_loss: 0.6290 - val_acc: 0.6360
Epoch 16/120
10000/10000 [==============================] - 6s 625us/step - loss: 0.6160 - acc: 0.6444 - val_loss: 0.6281 - val_acc: 0.6300
Epoch 17/120
10000/10000 [==============================] - 7s 651us/step - loss: 0.6193 - acc: 0.6408 - val_loss: 0.6278 - val_acc: 0.6330
Epoch 18/120
10000/10000 [==============================] - 6s 626us/step - loss: 0.6149 - acc: 0.6463 - val_loss: 0.6311 - val_acc: 0.6240
Epoch 19/120
10000/10000 [=======

10000/10000 [==============================] - 6s 624us/step - loss: 0.3553 - acc: 0.8472 - val_loss: 0.5765 - val_acc: 0.7740
Epoch 71/120
10000/10000 [==============================] - 6s 625us/step - loss: 0.3487 - acc: 0.8505 - val_loss: 0.5448 - val_acc: 0.7730
Epoch 72/120
10000/10000 [==============================] - 6s 625us/step - loss: 0.3514 - acc: 0.8524 - val_loss: 0.5570 - val_acc: 0.7690
Epoch 73/120
10000/10000 [==============================] - 7s 660us/step - loss: 0.3481 - acc: 0.8535 - val_loss: 0.5816 - val_acc: 0.7770
Epoch 74/120
10000/10000 [==============================] - 6s 625us/step - loss: 0.3339 - acc: 0.8587 - val_loss: 0.5870 - val_acc: 0.7640
Epoch 75/120
10000/10000 [==============================] - 6s 626us/step - loss: 0.3415 - acc: 0.8539 - val_loss: 0.5651 - val_acc: 0.7680
Epoch 76/120
10000/10000 [==============================] - 6s 627us/step - loss: 0.3354 - acc: 0.8571 - val_loss: 0.5824 - val_acc: 0.7750
Epoch 77/120
10000/10000 [=======

# QA10 Indefinite Knowledge

In [24]:
loss, acc = LSTMmodel(i)
append('LSTM',loss,acc,i)
print('\n\n\n')
loss, acc = memoryNet(i)
append('Memory',loss,acc,i)
i+=1

C:\Users\Manik\Anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


vocab = ['.', '?', 'Bill', 'Fred', 'Is', 'Julie', 'Mary', 'back', 'bedroom', 'cinema', 'either', 'in', 'is', 'journeyed', 'kitchen', 'maybe', 'moved', 'no', 'office', 'or', 'park', 'school', 'the', 'to', 'travelled', 'went', 'yes']

x.shape = (10000, 89)

xq.shape = (10000, 6)

y.shape = (10000, 28)

story_maxlen, query_maxlen = 89, 6

An example tuple from this dataset=====> (['Fred', 'is', 'either', 'in', 'the', 'school', 'or', 'the', 'park', '.', 'Mary', 'went', 'back', 'to', 'the', 'office', '.'], ['Is', 'Mary', 'in', 'the', 'office', '?'], 'yes') 



RNN / Embed / Sent / Query = <class 'keras.layers.recurrent.LSTM'>, 50, 100, 100


Build model...


Training


Train on 9500 samples, validate on 500 samples
Epoch 1/40
9500/9500 [==============================] - 26s 3ms/step - loss: 1.2085 - acc: 0.4406 - val_loss: 1.0059 - val_acc: 0.4200
Epoch 2/40
9500/9500 [==============================] - 21s 2ms/step - loss: 1.0301 - acc: 0.4536 - val_loss: 1.0088 - val_acc: 0.4400
Epoch 3/40

Epoch 12/120
10000/10000 [==============================] - 7s 719us/step - loss: 0.8948 - acc: 0.4906 - val_loss: 0.9134 - val_acc: 0.4720
Epoch 13/120
10000/10000 [==============================] - 7s 695us/step - loss: 0.8937 - acc: 0.4815 - val_loss: 0.9097 - val_acc: 0.4870
Epoch 14/120
10000/10000 [==============================] - 7s 700us/step - loss: 0.8912 - acc: 0.4892 - val_loss: 0.9197 - val_acc: 0.4840
Epoch 15/120
10000/10000 [==============================] - 7s 706us/step - loss: 0.8880 - acc: 0.4863 - val_loss: 0.9162 - val_acc: 0.4520
Epoch 16/120
10000/10000 [==============================] - 7s 696us/step - loss: 0.8849 - acc: 0.4980 - val_loss: 0.9054 - val_acc: 0.4790
Epoch 17/120
10000/10000 [==============================] - 7s 700us/step - loss: 0.8760 - acc: 0.4979 - val_loss: 0.8989 - val_acc: 0.4690
Epoch 18/120
10000/10000 [==============================] - 7s 701us/step - loss: 0.8649 - acc: 0.5067 - val_loss: 0.8869 - val_acc: 0.4840
Epoch 19/120
10000/1

10000/10000 [==============================] - 7s 698us/step - loss: 0.6112 - acc: 0.6685 - val_loss: 0.6364 - val_acc: 0.6500
Epoch 71/120
10000/10000 [==============================] - 7s 733us/step - loss: 0.5977 - acc: 0.6765 - val_loss: 0.6671 - val_acc: 0.6530
Epoch 72/120
10000/10000 [==============================] - 7s 703us/step - loss: 0.5969 - acc: 0.6812 - val_loss: 0.6245 - val_acc: 0.6690
Epoch 73/120
10000/10000 [==============================] - 7s 700us/step - loss: 0.5916 - acc: 0.6810 - val_loss: 0.6297 - val_acc: 0.6760
Epoch 74/120
10000/10000 [==============================] - 7s 693us/step - loss: 0.5875 - acc: 0.6864 - val_loss: 0.6305 - val_acc: 0.6720
Epoch 75/120
10000/10000 [==============================] - 7s 703us/step - loss: 0.5775 - acc: 0.6896 - val_loss: 0.6370 - val_acc: 0.6750
Epoch 76/120
10000/10000 [==============================] - 7s 697us/step - loss: 0.5736 - acc: 0.7003 - val_loss: 0.6083 - val_acc: 0.6810
Epoch 77/120
10000/10000 [=======

# QA11 Basic Coreference

In [25]:
loss, acc = LSTMmodel(i)
append('LSTM',loss,acc,i)
print('\n\n\n')
loss, acc = memoryNet(i)
append('Memory',loss,acc,i)
i+=1

C:\Users\Manik\Anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


vocab = ['.', '?', 'After', 'Afterwards', 'Daniel', 'Following', 'John', 'Mary', 'Sandra', 'Then', 'Where', 'back', 'bathroom', 'bedroom', 'garden', 'hallway', 'he', 'is', 'journeyed', 'kitchen', 'moved', 'office', 'she', 'that', 'the', 'to', 'travelled', 'went']

x.shape = (10000, 76)

xq.shape = (10000, 4)

y.shape = (10000, 29)

story_maxlen, query_maxlen = 76, 4

An example tuple from this dataset=====> (['Mary', 'went', 'back', 'to', 'the', 'bathroom', '.', 'After', 'that', 'she', 'went', 'to', 'the', 'bedroom', '.'], ['Where', 'is', 'Mary', '?'], 'bedroom') 



RNN / Embed / Sent / Query = <class 'keras.layers.recurrent.LSTM'>, 50, 100, 100


Build model...


Training


Train on 9500 samples, validate on 500 samples
Epoch 1/40
9500/9500 [==============================] - 23s 2ms/step - loss: 1.9602 - acc: 0.1917 - val_loss: 1.5832 - val_acc: 0.3400
Epoch 2/40
9500/9500 [==============================] - 18s 2ms/step - loss: 1.3217 - acc: 0.6129 - val_loss: 1.0257 - val_acc: 0.728

10000/10000 [==============================] - 5s 530us/step - loss: 0.9969 - acc: 0.6534 - val_loss: 0.7797 - val_acc: 0.7500
Epoch 12/120
10000/10000 [==============================] - 5s 531us/step - loss: 0.9703 - acc: 0.6597 - val_loss: 0.7572 - val_acc: 0.7530
Epoch 13/120
10000/10000 [==============================] - 5s 537us/step - loss: 0.9374 - acc: 0.6599 - val_loss: 0.7290 - val_acc: 0.7500
Epoch 14/120
10000/10000 [==============================] - 5s 534us/step - loss: 0.9160 - acc: 0.6737 - val_loss: 0.7064 - val_acc: 0.7460
Epoch 15/120
10000/10000 [==============================] - 5s 532us/step - loss: 0.9018 - acc: 0.6642 - val_loss: 0.6944 - val_acc: 0.7460
Epoch 16/120
10000/10000 [==============================] - 5s 531us/step - loss: 0.8725 - acc: 0.6749 - val_loss: 0.7030 - val_acc: 0.7310
Epoch 17/120
10000/10000 [==============================] - 6s 560us/step - loss: 0.8513 - acc: 0.6849 - val_loss: 0.6832 - val_acc: 0.7410
Epoch 18/120
10000/10000 [=======

10000/10000 [==============================] - 5s 531us/step - loss: 0.1696 - acc: 0.9451 - val_loss: 0.1366 - val_acc: 0.9580
Epoch 70/120
10000/10000 [==============================] - 5s 533us/step - loss: 0.1718 - acc: 0.9464 - val_loss: 0.1296 - val_acc: 0.9590
Epoch 71/120
10000/10000 [==============================] - 6s 563us/step - loss: 0.1627 - acc: 0.9467 - val_loss: 0.1334 - val_acc: 0.9570
Epoch 72/120
10000/10000 [==============================] - 5s 538us/step - loss: 0.1568 - acc: 0.9470 - val_loss: 0.1176 - val_acc: 0.9630
Epoch 73/120
10000/10000 [==============================] - 5s 533us/step - loss: 0.1597 - acc: 0.9491 - val_loss: 0.1184 - val_acc: 0.9610
Epoch 74/120
10000/10000 [==============================] - 5s 533us/step - loss: 0.1496 - acc: 0.9504 - val_loss: 0.1035 - val_acc: 0.9640
Epoch 75/120
10000/10000 [==============================] - 5s 533us/step - loss: 0.1324 - acc: 0.9548 - val_loss: 0.1073 - val_acc: 0.9620
Epoch 76/120
10000/10000 [=======

# QA12 Conjuction

In [30]:
i = 11

In [31]:
loss, acc = LSTMmodel(i)
append('LSTM',loss,acc,i)
print('\n\n\n')
loss, acc = memoryNet(i)
append('Memory',loss,acc,i)
i+=1

C:\Users\Manik\Anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


vocab = ['.', '?', 'Daniel', 'John', 'Mary', 'Sandra', 'Where', 'and', 'back', 'bathroom', 'bedroom', 'garden', 'hallway', 'is', 'journeyed', 'kitchen', 'moved', 'office', 'the', 'to', 'travelled', 'went']

x.shape = (10000, 87)

xq.shape = (10000, 4)

y.shape = (10000, 23)

story_maxlen, query_maxlen = 87, 4

An example tuple from this dataset=====> (['Mary', 'and', 'Daniel', 'travelled', 'to', 'the', 'bathroom', '.', 'John', 'and', 'Daniel', 'travelled', 'to', 'the', 'office', '.'], ['Where', 'is', 'Daniel', '?'], 'office') 



RNN / Embed / Sent / Query = <class 'keras.layers.recurrent.LSTM'>, 50, 100, 100


Build model...


Training


Train on 9500 samples, validate on 500 samples
Epoch 1/40
9500/9500 [==============================] - 27s 3ms/step - loss: 1.7897 - acc: 0.3055 - val_loss: 1.2005 - val_acc: 0.6080
Epoch 2/40
9500/9500 [==============================] - 21s 2ms/step - loss: 1.0137 - acc: 0.6943 - val_loss: 0.9088 - val_acc: 0.6800
Epoch 3/40
9500/9500 [==============

10000/10000 [==============================] - 6s 574us/step - loss: 0.9915 - acc: 0.7074 - val_loss: 0.8231 - val_acc: 0.7720
Epoch 13/120
10000/10000 [==============================] - 6s 579us/step - loss: 0.9750 - acc: 0.7130 - val_loss: 0.8190 - val_acc: 0.7720
Epoch 14/120
10000/10000 [==============================] - 6s 580us/step - loss: 0.9597 - acc: 0.7149 - val_loss: 0.8092 - val_acc: 0.7720
Epoch 15/120
10000/10000 [==============================] - 6s 580us/step - loss: 0.9565 - acc: 0.7117 - val_loss: 0.7933 - val_acc: 0.7720
Epoch 16/120
10000/10000 [==============================] - 6s 581us/step - loss: 0.9430 - acc: 0.7123 - val_loss: 0.8054 - val_acc: 0.7720
Epoch 17/120
10000/10000 [==============================] - 6s 586us/step - loss: 0.9371 - acc: 0.7142 - val_loss: 0.7890 - val_acc: 0.7720
Epoch 18/120
10000/10000 [==============================] - 6s 579us/step - loss: 0.9330 - acc: 0.7099 - val_loss: 0.7829 - val_acc: 0.7720
Epoch 19/120
10000/10000 [=======

10000/10000 [==============================] - 6s 600us/step - loss: 0.1994 - acc: 0.9490 - val_loss: 0.1439 - val_acc: 0.9640
Epoch 71/120
10000/10000 [==============================] - 6s 579us/step - loss: 0.1832 - acc: 0.9514 - val_loss: 0.1513 - val_acc: 0.9620
Epoch 72/120
10000/10000 [==============================] - 6s 581us/step - loss: 0.1862 - acc: 0.9536 - val_loss: 0.1388 - val_acc: 0.9610
Epoch 73/120
10000/10000 [==============================] - 6s 580us/step - loss: 0.1704 - acc: 0.9554 - val_loss: 0.1397 - val_acc: 0.9620
Epoch 74/120
10000/10000 [==============================] - 6s 577us/step - loss: 0.1679 - acc: 0.9561 - val_loss: 0.1388 - val_acc: 0.9620
Epoch 75/120
10000/10000 [==============================] - 6s 581us/step - loss: 0.1639 - acc: 0.9580 - val_loss: 0.1343 - val_acc: 0.9650
Epoch 76/120
10000/10000 [==============================] - 6s 581us/step - loss: 0.1656 - acc: 0.9563 - val_loss: 0.1144 - val_acc: 0.9660
Epoch 77/120
10000/10000 [=======

# QA13 Compound Coreference

In [32]:
loss, acc = LSTMmodel(i)
append('LSTM',loss,acc,i)
print('\n\n\n')
loss, acc = memoryNet(i)
append('Memory',loss,acc,i)
i+=1

C:\Users\Manik\Anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


vocab = ['.', '?', 'After', 'Afterwards', 'Daniel', 'Following', 'John', 'Mary', 'Sandra', 'Then', 'Where', 'and', 'back', 'bathroom', 'bedroom', 'garden', 'hallway', 'is', 'journeyed', 'kitchen', 'moved', 'office', 'that', 'the', 'they', 'to', 'travelled', 'went']

x.shape = (10000, 86)

xq.shape = (10000, 4)

y.shape = (10000, 29)

story_maxlen, query_maxlen = 86, 4

An example tuple from this dataset=====> (['Mary', 'and', 'Daniel', 'went', 'to', 'the', 'bathroom', '.', 'Then', 'they', 'journeyed', 'to', 'the', 'hallway', '.'], ['Where', 'is', 'Daniel', '?'], 'hallway') 



RNN / Embed / Sent / Query = <class 'keras.layers.recurrent.LSTM'>, 50, 100, 100


Build model...


Training


Train on 9500 samples, validate on 500 samples
Epoch 1/40
9500/9500 [==============================] - 26s 3ms/step - loss: 1.7233 - acc: 0.3499 - val_loss: 0.7946 - val_acc: 0.9400
Epoch 2/40
9500/9500 [==============================] - 20s 2ms/step - loss: 0.5418 - acc: 0.9192 - val_loss: 0.3459 - val_

10000/10000 [==============================] - 6s 573us/step - loss: 0.4032 - acc: 0.9289 - val_loss: 0.3062 - val_acc: 0.9440
Epoch 12/120
10000/10000 [==============================] - 6s 580us/step - loss: 0.4023 - acc: 0.9298 - val_loss: 0.3060 - val_acc: 0.9440
Epoch 13/120
10000/10000 [==============================] - 6s 582us/step - loss: 0.4008 - acc: 0.9306 - val_loss: 0.3053 - val_acc: 0.9440
Epoch 14/120
10000/10000 [==============================] - 6s 582us/step - loss: 0.4012 - acc: 0.9305 - val_loss: 0.3051 - val_acc: 0.9440
Epoch 15/120
10000/10000 [==============================] - 6s 612us/step - loss: 0.3901 - acc: 0.9312 - val_loss: 0.3023 - val_acc: 0.9440
Epoch 16/120
10000/10000 [==============================] - 6s 581us/step - loss: 0.3882 - acc: 0.9320 - val_loss: 0.3074 - val_acc: 0.9440
Epoch 17/120
10000/10000 [==============================] - 6s 584us/step - loss: 0.3877 - acc: 0.9324 - val_loss: 0.3051 - val_acc: 0.9440
Epoch 18/120
10000/10000 [=======

10000/10000 [==============================] - 6s 575us/step - loss: 0.3166 - acc: 0.9349 - val_loss: 0.2958 - val_acc: 0.9440
Epoch 70/120
10000/10000 [==============================] - 6s 590us/step - loss: 0.3159 - acc: 0.9347 - val_loss: 0.2963 - val_acc: 0.9450
Epoch 71/120
10000/10000 [==============================] - 6s 579us/step - loss: 0.3111 - acc: 0.9333 - val_loss: 0.3137 - val_acc: 0.9320
Epoch 72/120
10000/10000 [==============================] - 6s 586us/step - loss: 0.3153 - acc: 0.9334 - val_loss: 0.2981 - val_acc: 0.9430
Epoch 73/120
10000/10000 [==============================] - 6s 591us/step - loss: 0.3133 - acc: 0.9345 - val_loss: 0.3041 - val_acc: 0.9420
Epoch 74/120
10000/10000 [==============================] - 6s 580us/step - loss: 0.3048 - acc: 0.9353 - val_loss: 0.3041 - val_acc: 0.9450
Epoch 75/120
10000/10000 [==============================] - 6s 577us/step - loss: 0.3101 - acc: 0.9336 - val_loss: 0.3057 - val_acc: 0.9410
Epoch 76/120
10000/10000 [=======

# QA14 Time Reasoning

In [33]:
loss, acc = LSTMmodel(i)
append('LSTM',loss,acc,i)
print('\n\n\n')
loss, acc = memoryNet(i)
append('Memory',loss,acc,i)
i+=1

C:\Users\Manik\Anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


vocab = ['.', '?', 'Bill', 'Fred', 'Julie', 'Mary', 'This', 'Where', 'Yesterday', 'afternoon', 'back', 'bedroom', 'before', 'cinema', 'evening', 'journeyed', 'kitchen', 'morning', 'moved', 'office', 'park', 'school', 'the', 'this', 'to', 'travelled', 'was', 'went', 'yesterday']

x.shape = (10000, 116)

xq.shape = (10000, 7)

y.shape = (10000, 30)

story_maxlen, query_maxlen = 116, 7

An example tuple from this dataset=====> (['Bill', 'went', 'back', 'to', 'the', 'cinema', 'yesterday', '.', 'Julie', 'went', 'to', 'the', 'school', 'this', 'morning', '.', 'Fred', 'went', 'to', 'the', 'park', 'yesterday', '.', 'Yesterday', 'Julie', 'went', 'to', 'the', 'office', '.'], ['Where', 'was', 'Julie', 'before', 'the', 'school', '?'], 'office') 



RNN / Embed / Sent / Query = <class 'keras.layers.recurrent.LSTM'>, 50, 100, 100


Build model...


Training


Train on 9500 samples, validate on 500 samples
Epoch 1/40
9500/9500 [==============================] - 33s 3ms/step - loss: 1.9951 - acc: 0.164

10000/10000 [==============================] - 8s 783us/step - loss: 1.5420 - acc: 0.3275 - val_loss: 1.5034 - val_acc: 0.3360
Epoch 11/120
10000/10000 [==============================] - 8s 782us/step - loss: 1.5319 - acc: 0.3386 - val_loss: 1.4970 - val_acc: 0.3610
Epoch 12/120
10000/10000 [==============================] - 8s 788us/step - loss: 1.5217 - acc: 0.3449 - val_loss: 1.4815 - val_acc: 0.3480
Epoch 13/120
10000/10000 [==============================] - 8s 781us/step - loss: 1.4993 - acc: 0.3623 - val_loss: 1.4682 - val_acc: 0.3720
Epoch 14/120
10000/10000 [==============================] - 8s 793us/step - loss: 1.4852 - acc: 0.3721 - val_loss: 1.4582 - val_acc: 0.3640
Epoch 15/120
10000/10000 [==============================] - 8s 803us/step - loss: 1.4818 - acc: 0.3834 - val_loss: 1.4579 - val_acc: 0.3800
Epoch 16/120
10000/10000 [==============================] - 8s 776us/step - loss: 1.4676 - acc: 0.3851 - val_loss: 1.4418 - val_acc: 0.3780
Epoch 17/120
10000/10000 [=======

10000/10000 [==============================] - 8s 805us/step - loss: 1.1588 - acc: 0.5358 - val_loss: 1.3176 - val_acc: 0.4550
Epoch 69/120
10000/10000 [==============================] - 8s 783us/step - loss: 1.1564 - acc: 0.5379 - val_loss: 1.3355 - val_acc: 0.4490
Epoch 70/120
10000/10000 [==============================] - 8s 783us/step - loss: 1.1508 - acc: 0.5444 - val_loss: 1.3217 - val_acc: 0.4510
Epoch 71/120
10000/10000 [==============================] - 8s 778us/step - loss: 1.1489 - acc: 0.5472 - val_loss: 1.3250 - val_acc: 0.4480
Epoch 72/120
10000/10000 [==============================] - 8s 785us/step - loss: 1.1422 - acc: 0.5441 - val_loss: 1.3242 - val_acc: 0.4400
Epoch 73/120
10000/10000 [==============================] - 8s 793us/step - loss: 1.1378 - acc: 0.5545 - val_loss: 1.3068 - val_acc: 0.4380
Epoch 74/120
10000/10000 [==============================] - 8s 781us/step - loss: 1.1442 - acc: 0.5420 - val_loss: 1.3235 - val_acc: 0.4510
Epoch 75/120
10000/10000 [=======

# QA15 Basic Deduction

In [34]:
loss, acc = LSTMmodel(i)
append('LSTM',loss,acc,i)
print('\n\n\n')
loss, acc = memoryNet(i)
append('Memory',loss,acc,i)
i+=1

C:\Users\Manik\Anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


vocab = ['.', '?', 'Cats', 'Emily', 'Gertrude', 'Jessica', 'Mice', 'Sheep', 'What', 'Winona', 'Wolves', 'a', 'afraid', 'are', 'cat', 'cats', 'emily', 'gertrude', 'is', 'jessica', 'mice', 'mouse', 'of', 'sheep', 'winona', 'wolf', 'wolves']

x.shape = (10000, 44)

xq.shape = (10000, 6)

y.shape = (10000, 28)

story_maxlen, query_maxlen = 44, 6

An example tuple from this dataset=====> (['Mice', 'are', 'afraid', 'of', 'wolves', '.', 'Gertrude', 'is', 'a', 'mouse', '.', 'Cats', 'are', 'afraid', 'of', 'sheep', '.', 'Winona', 'is', 'a', 'mouse', '.', 'Sheep', 'are', 'afraid', 'of', 'wolves', '.', 'Wolves', 'are', 'afraid', 'of', 'cats', '.', 'Emily', 'is', 'a', 'mouse', '.', 'Jessica', 'is', 'a', 'wolf', '.'], ['What', 'is', 'gertrude', 'afraid', 'of', '?'], 'wolf') 



RNN / Embed / Sent / Query = <class 'keras.layers.recurrent.LSTM'>, 50, 100, 100


Build model...


Training


Train on 9500 samples, validate on 500 samples
Epoch 1/40
9500/9500 [==============================] - 17s 2ms/ste

10000/10000 [==============================] - 5s 492us/step - loss: 1.0881 - acc: 0.5332 - val_loss: 1.0412 - val_acc: 0.5390
Epoch 11/120
10000/10000 [==============================] - 5s 503us/step - loss: 1.0741 - acc: 0.5406 - val_loss: 1.0486 - val_acc: 0.5560
Epoch 12/120
10000/10000 [==============================] - 5s 490us/step - loss: 1.0455 - acc: 0.5454 - val_loss: 1.0227 - val_acc: 0.5480
Epoch 13/120
10000/10000 [==============================] - 5s 489us/step - loss: 1.0316 - acc: 0.5455 - val_loss: 0.9613 - val_acc: 0.5450
Epoch 14/120
10000/10000 [==============================] - 5s 495us/step - loss: 1.0228 - acc: 0.5430 - val_loss: 0.9469 - val_acc: 0.5770
Epoch 15/120
10000/10000 [==============================] - 5s 496us/step - loss: 1.0093 - acc: 0.5458 - val_loss: 0.9579 - val_acc: 0.5540
Epoch 16/120
10000/10000 [==============================] - 5s 494us/step - loss: 1.0007 - acc: 0.5502 - val_loss: 0.9330 - val_acc: 0.5600
Epoch 17/120
10000/10000 [=======

10000/10000 [==============================] - 5s 491us/step - loss: 0.8440 - acc: 0.6007 - val_loss: 0.9525 - val_acc: 0.5720
Epoch 69/120
10000/10000 [==============================] - 5s 493us/step - loss: 0.8384 - acc: 0.6005 - val_loss: 0.9263 - val_acc: 0.5800
Epoch 70/120
10000/10000 [==============================] - 5s 490us/step - loss: 0.8359 - acc: 0.6081 - val_loss: 0.9435 - val_acc: 0.5660
Epoch 71/120
10000/10000 [==============================] - 5s 487us/step - loss: 0.8380 - acc: 0.6074 - val_loss: 0.9350 - val_acc: 0.5730
Epoch 72/120
10000/10000 [==============================] - 5s 492us/step - loss: 0.8353 - acc: 0.6016 - val_loss: 0.9172 - val_acc: 0.5550
Epoch 73/120
10000/10000 [==============================] - 5s 493us/step - loss: 0.8282 - acc: 0.6038 - val_loss: 0.9177 - val_acc: 0.5680
Epoch 74/120
10000/10000 [==============================] - 5s 493us/step - loss: 0.8376 - acc: 0.6077 - val_loss: 0.9506 - val_acc: 0.5670
Epoch 75/120
10000/10000 [=======

# QA16 Basic Induction

In [36]:
i = 15

In [37]:
loss, acc = LSTMmodel(i)
append('LSTM',loss,acc,i)
print('\n\n\n')
loss, acc = memoryNet(i)
append('Memory',loss,acc,i)
i+=1

C:\Users\Manik\Anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


vocab = ['.', '?', 'Bernhard', 'Brian', 'Greg', 'Julius', 'Lily', 'What', 'a', 'color', 'frog', 'gray', 'green', 'is', 'lion', 'rhino', 'swan', 'white', 'yellow']

x.shape = (10000, 41)

xq.shape = (10000, 5)

y.shape = (10000, 20)

story_maxlen, query_maxlen = 41, 5

An example tuple from this dataset=====> (['Lily', 'is', 'a', 'frog', '.', 'Bernhard', 'is', 'a', 'frog', '.', 'Bernhard', 'is', 'green', '.', 'Brian', 'is', 'a', 'lion', '.', 'Brian', 'is', 'white', '.', 'Julius', 'is', 'a', 'swan', '.', 'Julius', 'is', 'green', '.', 'Lily', 'is', 'green', '.', 'Greg', 'is', 'a', 'swan', '.'], ['What', 'color', 'is', 'Greg', '?'], 'green') 



RNN / Embed / Sent / Query = <class 'keras.layers.recurrent.LSTM'>, 50, 100, 100


Build model...


Training


Train on 9500 samples, validate on 500 samples
Epoch 1/40
9500/9500 [==============================] - 20s 2ms/step - loss: 1.5560 - acc: 0.2560 - val_loss: 1.3929 - val_acc: 0.3440
Epoch 2/40
9500/9500 [==============================] - 1

Epoch 12/120
10000/10000 [==============================] - 4s 440us/step - loss: 1.1424 - acc: 0.4717 - val_loss: 1.0661 - val_acc: 0.4940
Epoch 13/120
10000/10000 [==============================] - 4s 441us/step - loss: 1.1059 - acc: 0.4841 - val_loss: 1.0556 - val_acc: 0.4900
Epoch 14/120
10000/10000 [==============================] - 4s 441us/step - loss: 1.0976 - acc: 0.4702 - val_loss: 1.0165 - val_acc: 0.5000
Epoch 15/120
10000/10000 [==============================] - 5s 462us/step - loss: 1.0810 - acc: 0.4754 - val_loss: 1.0044 - val_acc: 0.4880
Epoch 16/120
10000/10000 [==============================] - 5s 460us/step - loss: 1.0679 - acc: 0.4717 - val_loss: 1.0231 - val_acc: 0.5040
Epoch 17/120
10000/10000 [==============================] - 4s 442us/step - loss: 1.0602 - acc: 0.4721 - val_loss: 0.9761 - val_acc: 0.5030
Epoch 18/120
10000/10000 [==============================] - 4s 444us/step - loss: 1.0493 - acc: 0.4732 - val_loss: 0.9761 - val_acc: 0.4800
Epoch 19/120
10000/1

10000/10000 [==============================] - 4s 446us/step - loss: 0.8783 - acc: 0.5071 - val_loss: 0.8731 - val_acc: 0.4790
Epoch 71/120
10000/10000 [==============================] - 5s 450us/step - loss: 0.8838 - acc: 0.5105 - val_loss: 0.8737 - val_acc: 0.4900
Epoch 72/120
10000/10000 [==============================] - 4s 447us/step - loss: 0.8765 - acc: 0.5086 - val_loss: 0.8796 - val_acc: 0.4860
Epoch 73/120
10000/10000 [==============================] - 5s 455us/step - loss: 0.8782 - acc: 0.5110 - val_loss: 0.8763 - val_acc: 0.4900
Epoch 74/120
10000/10000 [==============================] - 5s 492us/step - loss: 0.8755 - acc: 0.5160 - val_loss: 0.8711 - val_acc: 0.4950
Epoch 75/120
10000/10000 [==============================] - 5s 455us/step - loss: 0.8749 - acc: 0.5229 - val_loss: 0.8703 - val_acc: 0.4910
Epoch 76/120
10000/10000 [==============================] - 5s 457us/step - loss: 0.8743 - acc: 0.5180 - val_loss: 0.8749 - val_acc: 0.4970
Epoch 77/120
10000/10000 [=======

# QA17 Positional Reasoning

In [38]:
loss, acc = LSTMmodel(i)
append('LSTM',loss,acc,i)
print('\n\n\n')
loss, acc = memoryNet(i)
append('Memory',loss,acc,i)
i+=1

C:\Users\Manik\Anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


vocab = ['.', '?', 'Is', 'The', 'above', 'below', 'blue', 'is', 'left', 'no', 'of', 'pink', 'rectangle', 'red', 'right', 'sphere', 'square', 'the', 'to', 'triangle', 'yellow', 'yes']

x.shape = (10000, 24)

xq.shape = (10000, 12)

y.shape = (10000, 23)

story_maxlen, query_maxlen = 24, 12

An example tuple from this dataset=====> (['The', 'triangle', 'is', 'above', 'the', 'pink', 'rectangle', '.', 'The', 'blue', 'square', 'is', 'to', 'the', 'left', 'of', 'the', 'triangle', '.'], ['Is', 'the', 'pink', 'rectangle', 'to', 'the', 'right', 'of', 'the', 'blue', 'square', '?'], 'yes') 



RNN / Embed / Sent / Query = <class 'keras.layers.recurrent.LSTM'>, 50, 100, 100


Build model...


Training


Train on 9500 samples, validate on 500 samples
Epoch 1/40
9500/9500 [==============================] - 17s 2ms/step - loss: 0.8935 - acc: 0.4980 - val_loss: 0.7161 - val_acc: 0.4440
Epoch 2/40
9500/9500 [==============================] - 9s 971us/step - loss: 0.7137 - acc: 0.4988 - val_loss: 0.7084 

10000/10000 [==============================] - 6s 600us/step - loss: 0.5927 - acc: 0.6145 - val_loss: 0.6411 - val_acc: 0.5740
Epoch 13/120
10000/10000 [==============================] - 6s 606us/step - loss: 0.5932 - acc: 0.6158 - val_loss: 0.6045 - val_acc: 0.5970
Epoch 14/120
10000/10000 [==============================] - 6s 618us/step - loss: 0.5898 - acc: 0.6209 - val_loss: 0.5999 - val_acc: 0.6220
Epoch 15/120
10000/10000 [==============================] - 6s 621us/step - loss: 0.5906 - acc: 0.6054 - val_loss: 0.5976 - val_acc: 0.6170
Epoch 16/120
10000/10000 [==============================] - 6s 621us/step - loss: 0.5890 - acc: 0.6196 - val_loss: 0.5994 - val_acc: 0.5910
Epoch 17/120
10000/10000 [==============================] - 6s 630us/step - loss: 0.5895 - acc: 0.6156 - val_loss: 0.5984 - val_acc: 0.6080
Epoch 18/120
10000/10000 [==============================] - 6s 621us/step - loss: 0.5907 - acc: 0.6162 - val_loss: 0.5981 - val_acc: 0.6100
Epoch 19/120
10000/10000 [=======

10000/10000 [==============================] - 6s 592us/step - loss: 0.5708 - acc: 0.6497 - val_loss: 0.6348 - val_acc: 0.5850
Epoch 71/120
10000/10000 [==============================] - 6s 584us/step - loss: 0.5711 - acc: 0.6542 - val_loss: 0.6196 - val_acc: 0.5960
Epoch 72/120
10000/10000 [==============================] - 6s 585us/step - loss: 0.5693 - acc: 0.6558 - val_loss: 0.6257 - val_acc: 0.5880
Epoch 73/120
10000/10000 [==============================] - 6s 588us/step - loss: 0.5671 - acc: 0.6549 - val_loss: 0.6266 - val_acc: 0.6020
Epoch 74/120
10000/10000 [==============================] - 6s 598us/step - loss: 0.5665 - acc: 0.6538 - val_loss: 0.6244 - val_acc: 0.6010
Epoch 75/120
10000/10000 [==============================] - 6s 596us/step - loss: 0.5664 - acc: 0.6596 - val_loss: 0.6321 - val_acc: 0.6030
Epoch 76/120
10000/10000 [==============================] - 6s 590us/step - loss: 0.5648 - acc: 0.6561 - val_loss: 0.6312 - val_acc: 0.6160
Epoch 77/120
10000/10000 [=======

# QA 18 Size Reasoning

In [39]:
loss, acc = LSTMmodel(i)
append('LSTM',loss,acc,i)
print('\n\n\n')
loss, acc = memoryNet(i)
append('Memory',loss,acc,i)
i+=1

C:\Users\Manik\Anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


vocab = ['.', '?', 'Does', 'Is', 'The', 'bigger', 'box', 'chest', 'chocolate', 'chocolates', 'container', 'fit', 'fits', 'in', 'inside', 'is', 'no', 'of', 'suitcase', 'than', 'the', 'yes']

x.shape = (10000, 169)

xq.shape = (10000, 10)

y.shape = (10000, 23)

story_maxlen, query_maxlen = 169, 10

An example tuple from this dataset=====> (['The', 'box', 'of', 'chocolates', 'fits', 'inside', 'the', 'chest', '.', 'The', 'box', 'is', 'bigger', 'than', 'the', 'chest', '.', 'The', 'box', 'is', 'bigger', 'than', 'the', 'suitcase', '.', 'The', 'suitcase', 'fits', 'inside', 'the', 'box', '.', 'The', 'container', 'is', 'bigger', 'than', 'the', 'box', 'of', 'chocolates', '.'], ['Does', 'the', 'box', 'fit', 'in', 'the', 'box', 'of', 'chocolates', '?'], 'no') 



RNN / Embed / Sent / Query = <class 'keras.layers.recurrent.LSTM'>, 50, 100, 100


Build model...


Training


Train on 9500 samples, validate on 500 samples
Epoch 1/40
9500/9500 [==============================] - 45s 5ms/step - loss: 0.8

Epoch 11/120
10000/10000 [==============================] - 13s 1ms/step - loss: 0.1162 - acc: 0.9312 - val_loss: 0.1480 - val_acc: 0.8930
Epoch 12/120
10000/10000 [==============================] - 13s 1ms/step - loss: 0.1160 - acc: 0.9313 - val_loss: 0.1569 - val_acc: 0.8930
Epoch 13/120
10000/10000 [==============================] - 13s 1ms/step - loss: 0.1150 - acc: 0.9309 - val_loss: 0.1415 - val_acc: 0.9130
Epoch 14/120
10000/10000 [==============================] - 13s 1ms/step - loss: 0.1117 - acc: 0.9335 - val_loss: 0.1761 - val_acc: 0.9020
Epoch 15/120
10000/10000 [==============================] - 13s 1ms/step - loss: 0.1102 - acc: 0.9365 - val_loss: 0.1595 - val_acc: 0.9130
Epoch 16/120
10000/10000 [==============================] - 13s 1ms/step - loss: 0.1110 - acc: 0.9390 - val_loss: 0.1511 - val_acc: 0.9090
Epoch 17/120
10000/10000 [==============================] - 13s 1ms/step - loss: 0.1057 - acc: 0.9385 - val_loss: 0.1472 - val_acc: 0.9120
Epoch 18/120
10000/10000 [=

Epoch 70/120
10000/10000 [==============================] - 13s 1ms/step - loss: 0.0516 - acc: 0.9802 - val_loss: 0.2137 - val_acc: 0.913015 - acc: 0
Epoch 71/120
10000/10000 [==============================] - 13s 1ms/step - loss: 0.0498 - acc: 0.9794 - val_loss: 0.2286 - val_acc: 0.9200
Epoch 72/120
10000/10000 [==============================] - 13s 1ms/step - loss: 0.0486 - acc: 0.9771 - val_loss: 0.2771 - val_acc: 0.9120
Epoch 73/120
10000/10000 [==============================] - 13s 1ms/step - loss: 0.0495 - acc: 0.9812 - val_loss: 0.2243 - val_acc: 0.9150
Epoch 74/120
10000/10000 [==============================] - 13s 1ms/step - loss: 0.0497 - acc: 0.9804 - val_loss: 0.2552 - val_acc: 0.9150
Epoch 75/120
10000/10000 [==============================] - 13s 1ms/step - loss: 0.0487 - acc: 0.9803 - val_loss: 0.1925 - val_acc: 0.9270
Epoch 76/120
10000/10000 [==============================] - 13s 1ms/step - loss: 0.0492 - acc: 0.9811 - val_loss: 0.2234 - val_acc: 0.9230
Epoch 77/120
100

# QA19 Path Finding

In [40]:
loss, acc = LSTMmodel(i)
append('LSTM',loss,acc,i)
print('\n\n\n')
loss, acc = memoryNet(i)
append('Memory',loss,acc,i)
i+=1

C:\Users\Manik\Anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


vocab = ['.', '?', 'How', 'The', 'bathroom', 'bedroom', 'do', 'e,e', 'e,n', 'e,s', 'east', 'from', 'garden', 'go', 'hallway', 'is', 'kitchen', 'n,e', 'n,n', 'n,w', 'north', 'of', 'office', 's,e', 's,s', 's,w', 'south', 'the', 'to', 'w,n', 'w,s', 'w,w', 'west', 'you']

x.shape = (10000, 40)

xq.shape = (10000, 11)

y.shape = (10000, 35)

story_maxlen, query_maxlen = 40, 11

An example tuple from this dataset=====> (['The', 'office', 'is', 'east', 'of', 'the', 'hallway', '.', 'The', 'kitchen', 'is', 'north', 'of', 'the', 'office', '.', 'The', 'garden', 'is', 'west', 'of', 'the', 'bedroom', '.', 'The', 'office', 'is', 'west', 'of', 'the', 'garden', '.', 'The', 'bathroom', 'is', 'north', 'of', 'the', 'garden', '.'], ['How', 'do', 'you', 'go', 'from', 'the', 'kitchen', 'to', 'the', 'garden', '?'], 's,e') 



RNN / Embed / Sent / Query = <class 'keras.layers.recurrent.LSTM'>, 50, 100, 100


Build model...


Training


Train on 9500 samples, validate on 500 samples
Epoch 1/40
9500/9500 [=====

Epoch 10/120
10000/10000 [==============================] - 7s 740us/step - loss: 2.4541 - acc: 0.1222 - val_loss: 2.4628 - val_acc: 0.1060
Epoch 11/120
10000/10000 [==============================] - 7s 743us/step - loss: 2.4548 - acc: 0.1195 - val_loss: 2.4619 - val_acc: 0.1010
Epoch 12/120
10000/10000 [==============================] - 7s 712us/step - loss: 2.4509 - acc: 0.1255 - val_loss: 2.4658 - val_acc: 0.0940
Epoch 13/120
10000/10000 [==============================] - 7s 694us/step - loss: 2.4503 - acc: 0.1209 - val_loss: 2.4649 - val_acc: 0.0990
Epoch 14/120
10000/10000 [==============================] - 7s 744us/step - loss: 2.4514 - acc: 0.1195 - val_loss: 2.4654 - val_acc: 0.0950
Epoch 15/120
10000/10000 [==============================] - 7s 737us/step - loss: 2.4512 - acc: 0.1202 - val_loss: 2.4672 - val_acc: 0.0950
Epoch 16/120
10000/10000 [==============================] - 7s 722us/step - loss: 2.4474 - acc: 0.1274 - val_loss: 2.4668 - val_acc: 0.0940
Epoch 17/120
10000/1

10000/10000 [==============================] - 7s 670us/step - loss: 2.3477 - acc: 0.1725 - val_loss: 2.4480 - val_acc: 0.1120
Epoch 69/120
10000/10000 [==============================] - 7s 668us/step - loss: 2.3417 - acc: 0.1694 - val_loss: 2.4540 - val_acc: 0.1180
Epoch 70/120
10000/10000 [==============================] - 7s 666us/step - loss: 2.3432 - acc: 0.1714 - val_loss: 2.4456 - val_acc: 0.1220
Epoch 71/120
10000/10000 [==============================] - 7s 667us/step - loss: 2.3415 - acc: 0.1768 - val_loss: 2.4503 - val_acc: 0.1170
Epoch 72/120
10000/10000 [==============================] - 7s 702us/step - loss: 2.3413 - acc: 0.1737 - val_loss: 2.4484 - val_acc: 0.1200
Epoch 73/120
10000/10000 [==============================] - 7s 667us/step - loss: 2.3404 - acc: 0.1753 - val_loss: 2.4467 - val_acc: 0.1220
Epoch 74/120
10000/10000 [==============================] - 7s 671us/step - loss: 2.3397 - acc: 0.1741 - val_loss: 2.4506 - val_acc: 0.1150
Epoch 75/120
10000/10000 [=======

# QA20 Agents Motivation

In [41]:
loss, acc = LSTMmodel(i)
append('LSTM',loss,acc,i)
print('\n\n\n')
loss, acc = memoryNet(i)
append('Memory',loss,acc,i)
i+=1

C:\Users\Manik\Anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


vocab = ['.', '?', 'Antoine', 'Jason', 'Sumit', 'Where', 'Why', 'Yann', 'antoine', 'apple', 'back', 'bedroom', 'bored', 'did', 'football', 'garden', 'get', 'go', 'got', 'grabbed', 'hungry', 'is', 'jason', 'journeyed', 'kitchen', 'milk', 'moved', 'pajamas', 'picked', 'sumit', 'the', 'there', 'thirsty', 'tired', 'to', 'took', 'travelled', 'up', 'went', 'will', 'yann']

x.shape = (10000, 69)

xq.shape = (10000, 8)

y.shape = (10000, 42)

story_maxlen, query_maxlen = 69, 8

An example tuple from this dataset=====> (['Sumit', 'is', 'tired', '.'], ['Where', 'will', 'sumit', 'go', '?'], 'bedroom') 



RNN / Embed / Sent / Query = <class 'keras.layers.recurrent.LSTM'>, 50, 100, 100


Build model...


Training


Train on 9500 samples, validate on 500 samples
Epoch 1/40
9500/9500 [==============================] - 27s 3ms/step - loss: 1.4217 - acc: 0.4489 - val_loss: 0.6600 - val_acc: 0.6440
Epoch 2/40
9500/9500 [==============================] - 18s 2ms/step - loss: 0.6586 - acc: 0.6818 - val_l

Epoch 11/120
10000/10000 [==============================] - 7s 688us/step - loss: 0.1037 - acc: 0.9587 - val_loss: 0.0667 - val_acc: 0.9700
Epoch 12/120
10000/10000 [==============================] - 7s 691us/step - loss: 0.0973 - acc: 0.9595 - val_loss: 0.0696 - val_acc: 0.9700
Epoch 13/120
10000/10000 [==============================] - 7s 674us/step - loss: 0.0938 - acc: 0.9614 - val_loss: 0.0580 - val_acc: 0.9740
Epoch 14/120
10000/10000 [==============================] - 7s 680us/step - loss: 0.0890 - acc: 0.9639 - val_loss: 0.0562 - val_acc: 0.9740
Epoch 15/120
10000/10000 [==============================] - 7s 718us/step - loss: 0.0866 - acc: 0.9673 - val_loss: 0.0551 - val_acc: 0.9750
Epoch 16/120
10000/10000 [==============================] - 7s 678us/step - loss: 0.0915 - acc: 0.9653 - val_loss: 0.0609 - val_acc: 0.9730
Epoch 17/120
10000/10000 [==============================] - 7s 673us/step - loss: 0.0863 - acc: 0.9662 - val_loss: 0.0531 - val_acc: 0.9810
Epoch 18/120
10000/1

10000/10000 [==============================] - 7s 678us/step - loss: 0.0552 - acc: 0.9745 - val_loss: 0.0406 - val_acc: 0.9830
Epoch 70/120
10000/10000 [==============================] - 7s 709us/step - loss: 0.0555 - acc: 0.9762 - val_loss: 0.0425 - val_acc: 0.9820
Epoch 71/120
10000/10000 [==============================] - 7s 685us/step - loss: 0.0570 - acc: 0.9755 - val_loss: 0.0398 - val_acc: 0.9860
Epoch 72/120
10000/10000 [==============================] - 7s 689us/step - loss: 0.0571 - acc: 0.9755 - val_loss: 0.0428 - val_acc: 0.9850
Epoch 73/120
10000/10000 [==============================] - 7s 676us/step - loss: 0.0573 - acc: 0.9770 - val_loss: 0.0419 - val_acc: 0.9820
Epoch 74/120
10000/10000 [==============================] - 7s 710us/step - loss: 0.0601 - acc: 0.9747 - val_loss: 0.0424 - val_acc: 0.9840
Epoch 75/120
10000/10000 [==============================] - 7s 676us/step - loss: 0.0588 - acc: 0.9780 - val_loss: 0.0410 - val_acc: 0.9850
Epoch 76/120
10000/10000 [=======

# LSTM Model Result

In [43]:
resultsLSTM.head(20)

,QA Dataset,Loss,Accuracy
0,QA1: Single supporting fact,0.042328,0.994
1,QA2:Two supporting facts,1.330328,0.430
2,QA3: Three supporting facts,1.621032,0.192
3,QA4: Two arg relations,0.335323,0.773
4,QA5: Three arg relations,0.061982,0.986
5,QA6: Yes No Questions,0.693864,0.503
6,QA7: Counting,0.397963,0.812
7,QA8: Lists Sets,0.499944,0.767
8,QA9: Simple Negation,0.344357,0.847
9,QA10: Indefinite Knowledge,0.697126,0.501


# Memory Model Result

In [44]:
resultsMem.head(20)

,QA Dataset,Loss,Accuracy
0,QA1: Single supporting fact,0.250331,0.916
1,QA2:Two supporting facts,2.144747,0.314
2,QA3: Three supporting facts,2.712857,0.243
3,QA4: Two arg relations,0.327017,0.789
4,QA5: Three arg relations,0.485006,0.873
5,QA6: Yes No Questions,0.473377,0.820
6,QA7: Counting,0.667514,0.786
7,QA8: Lists Sets,0.849950,0.761
8,QA9: Simple Negation,0.690926,0.775
9,QA10: Indefinite Knowledge,0.303344,0.866
